In [1]:
LANGCHAIN_API_KEY = "ls__ef7c69da49cf48719e4d7c4b33a3debd"
import os
os.environ['LANGCHAIN_API_KEY'] = LANGCHAIN_API_KEY

In [2]:
import json
with open("train.json", "r") as file:
    dataset = json.load(file)
file.close()

In [3]:
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.document import Document

In [4]:
documents = []
document_labels = []
for data_part in dataset[1:100]:
    for data in data_part['annotations'][0]['result']:
        documents.append(Document(data['value']['text']))
        document_labels.append(data['value']['labels'])

In [5]:
len(document_labels)

11529

In [6]:
doc_label = {}
for i, doc in enumerate(documents):
    doc_label[doc.page_content] = document_labels[i]

In [7]:
from langchain_community.embeddings import GPT4AllEmbeddings
embeddings = GPT4AllEmbeddings()

In [8]:
# vector = Chroma.from_documents(documents=documents, embedding=embeddings, persist_directory='./embedding')
vector = Chroma(persist_directory='./embedding', embedding_function=embeddings)

In [9]:
from tqdm import tqdm

true_labels = []
pred_labels = []
for data in tqdm(dataset[0]['annotations'][0]['result']):
    sample_query = data['value']['text']
    
    true_labels.append(data['value']['labels'][0])
    query_labels = []
    output = vector.similarity_search(query=sample_query, k=5)

    for out in output:
        query_labels.append([out.page_content, doc_label[out.page_content][0]])
    pred_labels.append(query_labels)

100%|██████████| 91/91 [00:00<00:00, 100.68it/s]


In [10]:
len(true_labels), len(pred_labels)

(91, 91)

In [14]:
pred_labels[0], true_labels[0]

([['     IN THE HIGH COURT OF KARNATAKA\n                  DHARWAD BENCH\n\nDATED THIS THE 21ST DAY OF SEPTEMBER, 2015',
   'PREAMBLE'],
  ['IN THE HIGH COURT OF KARNATAKA AT BENGALURU\n\nDATED THIS THE 22ND DAY OF DECEMBER 2016\n\nBEFORE',
   'PREAMBLE'],
  ['IN THE HIGH COURT OF KARNATAKA AT BENGALURU\n         DATED THIS THE 13TH DAY OF JUNE 2019\n                                   BEFORE',
   'PREAMBLE'],
  ['IN THE HIGH COURT OF KARNATAKA AT BENGALURU\n\nDATED THIS THE 26TH DAY OF FEBRUARY 2016',
   'PREAMBLE'],
  ['IN THE HIGH COURT OF KARNATAKA AT BENGALURU\n    DATED THIS THE 29TH DAY OF JANUARY, 2015\n                                  BEFORE',
   'PREAMBLE']],
 'PREAMBLE')

In [15]:
import joblib
joblib.dump(pred_labels, './vecs/top_preds')
joblib.dump(true_labels, './vecs/true_labels')
joblib.dump(doc_label, "./vecs/doc_label")

['./vecs/doc_label']